### 📁 Available LoRa Models

| Model Size | Motion Type | LoRa Name | Description |
|-------|-------------|-----------|-------------|
| Lite, Pro | Object Rotation | `Microwave-right` | Rotates object ~360° to the right over 5 seconds |
| Lite, Pro | Object Rotation | `Microwave-left` | Rotates object ~360° to the left over 5 seconds |
| Lite, Pro | Camera Movement | `Arc-right` | Camera arcs ~360° around object to the right over 5 seconds |
| Lite, Pro | Camera Movement | `Arc-left` | Camera arcs ~360° around object to the left over 5 seconds |
| Lite | Camera Movement | `Dolly-in` | Camera moves forward toward subject |
| Lite | Camera Movement | `Dolly-out` | Camera moves backward from subject |
| Lite | Camera Movement | `Truck-right` | Camera tracks horizontally to the right |
| Lite | Camera Movement | `Truck-left` | Camera tracks horizontally to the left |

<br>

>  Note:
> 1. For optimal performance, select an appropriate LoRa for the **t2v** (text-to-video) or **i2v** (image-to-video) modality.  
>    We named our LoRas using the template: `kandinskylab/Kandinsky-5.0-{Modality}-{Model_Size}-LoRa-{LoRa_Name}` where `Modality` is either `I2V` or `T2V`.
> 2. Additional camera movements in the **Pro** model can be achieved directly via the prompt!

In [1]:
import os
os.chdir("..")

import torch
from huggingface_hub import snapshot_download
from kandinsky import get_T2V_pipeline

from IPython.display import Video

FlashAttention 2 is found
FlashAttention 3 is found


### Load pipe 
first, ensure that you have downloaded the models using `download_models.py`

In [2]:
pipe = get_T2V_pipeline(
    device_map={"dit": "cuda:0", "vae": "cuda:0", "text_embedder": "cuda:0"},
    conf_path="./configs/k5_lite_t2v_5s_sft_sd.yaml"
)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

The module name  (originally ) is not a valid Python identifier. Please rename the original module to avoid import issues.


#### Download LoRa adapter from HuggingFace🤗:

In [3]:
cache_dir = './weights'
lora_name = 'Microwave-right'
repo_id = f"kandinskylab/Kandinsky-5.0-T2V-Lite-LoRa-{lora_name}"
adapter_path = snapshot_download(
    repo_id=repo_id,
    local_dir=os.path.join(cache_dir, repo_id),
    token=None
)

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

config_lora.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

.gitattributes: 0.00B [00:00, ?B/s]

Load the adapter into the pipe:

In [4]:
pipe.load_adapter(
    adapter_config=os.path.join(cache_dir, repo_id, "config_lora.json"),
    adapter_path= os.path.join(cache_dir, repo_id, "lora.safetensors"),
    adapter_name=lora_name
)

Note: The trigger words we use are stored in the LoRA safetensors metadata. They are automatically concatenated with prompts during inference.

In [5]:
pipe.peft_triggers

{'Microwave-right': 'M1CR0W4V3 r0t4tION, 360-degree rotation. '}

Generate:

In [6]:
save_dir = './outputs/loras/'
os.makedirs(save_dir, exist_ok=True)
out = pipe(
    "a cat in a red hat",
    time_length=5,
    width=768,
    height=512,
    save_path=os.path.join(save_dir, f't2v_lite_{lora_name}.mp4'),
    seed=1234,
)
Video(os.path.join(save_dir, f't2v_lite_{lora_name}.mp4'))

100%|██████████| 50/50 [03:46<00:00,  4.53s/it] 
/home/jovyan/shares/SR008.fs2/me/envs/kandi5/lib/python3.12/site-packages/torchvision/io/_video_deprecation_warning.py:5: UserWarning: The video decoding and encoding capabilities of torchvision are deprecated from version 0.22 and will be removed in version 0.24. We recommend that you migrate to TorchCodec, where we'll consolidate the future decoding/encoding capabilities of PyTorch: https://github.com/pytorch/torchcodec
  warnings.warn(


Download another adapter and switch to it:

In [7]:
lora_name = 'Microwave-left'
repo_id = f"kandinskylab/Kandinsky-5.0-T2V-Lite-LoRa-{lora_name}"
adapter_path = snapshot_download(
    repo_id=repo_id,
    local_dir=os.path.join(cache_dir, repo_id),
    token=None
)
pipe.load_adapter(
    adapter_config=os.path.join(cache_dir, repo_id, "config_lora.json"),
    adapter_path= os.path.join(cache_dir, repo_id, "lora.safetensors"),
    adapter_name=lora_name
)

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
out = pipe(
    "a cat in a red hat",
    time_length=5,
    width=768,
    height=512,
    save_path=os.path.join(save_dir, f't2v_lite_{lora_name}.mp4'),
    seed=1234,
)
Video(os.path.join(save_dir, f't2v_lite_{lora_name}.mp4'))

100%|██████████| 50/50 [03:33<00:00,  4.26s/it]
/home/user/conda/envs/kandinsky-cuda12.8/lib/python3.12/site-packages/torchvision/io/_video_deprecation_warning.py:5: UserWarning: The video decoding and encoding capabilities of torchvision are deprecated from version 0.22 and will be removed in version 0.24. We recommend that you migrate to TorchCodec, where we'll consolidate the future decoding/encoding capabilities of PyTorch: https://github.com/pytorch/torchcodec
  warnings.warn(


Switch to the previous LoRa adapter:

In [9]:
prev_lora_name = 'Microwave-right'
pipe.set_adapter(prev_lora_name)

In [10]:
out = pipe(
    "a cat in a red hat",
    time_length=5,
    width=768,
    height=512,
    save_path=os.path.join(save_dir, f't2v_lite_{prev_lora_name}_v2.mp4'),
    seed=1234,
)
Video(os.path.join(save_dir, f't2v_lite_{prev_lora_name}_v2.mp4'))

100%|██████████| 50/50 [03:31<00:00,  4.23s/it]
/home/user/conda/envs/kandinsky-cuda12.8/lib/python3.12/site-packages/torchvision/io/_video_deprecation_warning.py:5: UserWarning: The video decoding and encoding capabilities of torchvision are deprecated from version 0.22 and will be removed in version 0.24. We recommend that you migrate to TorchCodec, where we'll consolidate the future decoding/encoding capabilities of PyTorch: https://github.com/pytorch/torchcodec
  warnings.warn(


Disable all adapters:

In [11]:
pipe.disable_adapters()

In [12]:
out = pipe(
    "a cat in a red hat",
    time_length=5,
    width=768,
    height=512,
    save_path=os.path.join(save_dir, f'default.mp4'),
    seed=1234,
)
Video(os.path.join(save_dir, f'default.mp4'))

100%|██████████| 50/50 [03:21<00:00,  4.03s/it]
/home/user/conda/envs/kandinsky-cuda12.8/lib/python3.12/site-packages/torchvision/io/_video_deprecation_warning.py:5: UserWarning: The video decoding and encoding capabilities of torchvision are deprecated from version 0.22 and will be removed in version 0.24. We recommend that you migrate to TorchCodec, where we'll consolidate the future decoding/encoding capabilities of PyTorch: https://github.com/pytorch/torchcodec
  warnings.warn(
